<a href="https://colab.research.google.com/github/returaj/cs6910/blob/assginment1_akash/pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/DLASSIGNMENT/optimizer.py .
import optimizer
!cp drive/MyDrive/DLASSIGNMENT/activation_func.py .
import activation_func

Mounted at /content/drive


In [7]:
from sklearn.metrics import accuracy_score
class BaseLossFunc(object):
  @staticmethod
  def error(X, y, model):
    raise NotImplementedError("error() method not implemented")

  @staticmethod
  def grad(layer, y):
    raise NotImplementedError("grad() method not implement")
class CrossEntropy(BaseLossFunc):
  @staticmethod
  def error(X, y, model):
    """
    X: (batch_size(B), data_size(N))
    y: (batch_size(B))
    """
    batch_size = X.shape[0]
    prob = model.forward(X)[-1]
    err = - np.sum(np.log(prob[np.arange(batch_size), y])) / batch_size
    for w in model.weight:
      err += model.reg * np.sum(w**2)
    return err
  @staticmethod
  def predict(X,Y_true,model):
    data_size=X.shape[0]
    prob=model.forward(X)[-1]
    Y_pred=[]
    iter=0
    for i in range(data_size):
      Y_pred.append(np.argmax(prob[i]))
    for j in range(data_size):
      if Y_pred[j]==Y_true[j]:
        iter=iter+1
    accuracy=iter/data_size
    accuracy=accuracy*100
    print(f'accuracy is:{accuracy}')
    return  Y_pred


    

    

  @staticmethod
  def grad(layer, y):
    """
    layer: (batch_size(B), output_size(O))
    y: (batch_size(B))
    """
    batch_size = layer.shape[0]
    dl = layer / batch_size
    dl[np.arange(batch_size), y] -= 1/batch_size
    return dl

In [8]:

import numpy as np
from keras.datasets import fashion_mnist
from optimizer import SGD
from activation_func import Sigmoid, Relu, Tanh
class FNN(object):
  def __init__(self, input_size, output_size, hidden_layers_size, act_func, loss_func, reg=0, init='random'):
    self.input_size = input_size
    self.output_size = output_size
    self.weight, self.bias = None, None
    self.act_func = act_func
    self.loss_func = loss_func
    self.reg = reg
    self.initialize(input_size, hidden_layers_size, output_size, init.lower())

  def initialize(self, input_size, hidden_layers_size, output_size, type):
    self.weight, self.bias = [], []
    prev_layer_size = input_size
    hidden_layers_size.append(output_size)
    for curr_layer_size in hidden_layers_size:
      std = np.sqrt(prev_layer_size * curr_layer_size) if type == 'xavier' else 1
      self.weight.append(np.random.rand(prev_layer_size, curr_layer_size)/std)
      self.bias.append(np.zeros(curr_layer_size))
      prev_layer_size = curr_layer_size

  @staticmethod
  def softmax(x):
    """
    x: (batch_size(B), data_size(N))
    """
    max_x = np.max(x, axis=1, keepdims=True)
    exp_prob = np.exp(x - max_x)
    prob = exp_prob / np.sum(exp_prob, axis=1, keepdims=True)
    return prob

  def forward(self, X):
    """
    X: (batch_size(B), data_size(N))
    """
    layer_output = []
    prev_layer = X
    num_hidden_layers = last_layer = len(self.weight) - 1
    for t in range(num_hidden_layers):
      w, b = self.weight[t], self.bias[t]
      next_layer = self.act_func.apply(np.dot(prev_layer, w) + b)
      layer_output.append(next_layer)
      prev_layer = next_layer
    w, b = self.weight[last_layer], self.bias[last_layer]
    prob = self.softmax(np.dot(prev_layer, w) + b)
    layer_output.append(prob)
    return layer_output

  def backward(self, X, y, layer_output):
    """
    X: (batch_size(B), data_size(N))
    y: (batch_size(B))
    """
    batch_size, _ = X.shape
    num_hidden_layers = last_layer = len(layer_output)-1
    dw, db = [None]*(num_hidden_layers+1), [None]*(num_hidden_layers+1)
    for t in range(num_hidden_layers, -1, -1):
      if t == last_layer:
        dh = self.loss_func.grad(layer_output[t], y)
      else:
        dh = np.dot(dh_fwd, self.weight[t+1].T) * self.act_func.grad(layer_output[t])
      prev_layer_output = X if t==0 else layer_output[t-1]
      dw[t] = np.dot(prev_layer_output.T, dh) + self.reg * self.weight[t]
      db[t] = np.sum(dh, axis=0)
      dh_fwd = dh
    return dw, db


if __name__ == '__main__':
  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
  consider =100
  X = np.array([(x_train[i].flatten())/255 for i in range(consider)]) / 255
  Y = y_train[:consider]
  batch_size, epochs = 16, 10
  act_func, loss_func = Sigmoid(), CrossEntropy()
  model = FNN(784, 10, [50, 20], act_func=act_func, loss_func=loss_func, init='random')
  sgd = SGD(model, 0.001)
  for ep in range(1, epochs+1):
    ids = np.arange(consider)
    np.random.shuffle(ids)
    start, end = 0, batch_size
    while end > start:
      x, y = X[ids[start:end]], Y[ids[start:end]]
      sgd.optimize(x, y)
      start, end = end, min(consider, end+batch_size)
    err = loss_func.error(X,Y,model)
    ##print(f'epoch: {ep}, error: {err}')
  Y_pred=loss_func.predict(X,Y,model)
  print(Y_pred)
 

accuracy is:12.0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
